In [2]:
import os
from pandas import DataFrame 
import networkx as nx

os.chdir("/home/yizhou7/mariposa/")

from debugger.debugger import Debugger3
from debugger.tree_node import NodeRef
from evaluator import BenchViewer
from evaluator import DebuggerStatus
from benchmark_consts import *
from utils.system_utils import list_smt2_files, get_name_hash

In [ ]:
qs = UNSTABLE_MARIPOSA
# qs = [q for q in qs if get_name_hash(q) not in LIKELY_FIXED]
# print(len(qs))
mv = BenchViewer(qs)
# vv = BenchViewer(UNSTABLE_VERUS)

In [4]:
mv.status.print_status()

| category                                      |   count | percentage   |
|-----------------------------------------------|---------|--------------|
| DebuggerStatus.FINISHED                       |     474 | 86.97 %      |
| DebuggerStatus.NO_PROOF                       |      26 | 4.77 %       |
| DebuggerStatus.SINGLETON_CREATION_FAILED      |      21 | 3.85 %       |
| DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED |      13 | 2.39 %       |
| DebuggerStatus.SINGLETON_NOT_RAN              |      10 | 1.83 %       |
| DebuggerStatus.FILTERED_NOT_RAN               |       1 | 0.18 %       |
| total                                         |     545 | 100.00 %     |


In [ ]:
for q in mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED]:
# for q in mv.status[DebuggerStatus.SINGLETON_CREATION_FAILED]:
    print("./src/debugger3.py --create-singleton -i", q, "--reset-proof-cache")

In [ ]:
for q in mv.status[DebuggerStatus.SINGLETON_NOT_RAN]:
# for q in mv.status[DebuggerStatus.SINGLETON_TESTED_EMPTY]:
    assert len(list_smt2_files(mv[q].singleton_dir)) != 0
    print("./src/exper_wizard.py manager -e verify --total-parts 12 -i ", mv[q].singleton_dir, "--clear")
    print("./src/analysis_wizard.py singleton -e verify -s z3_4_13_0 -i ", mv[q].singleton_dir)
    print("python3 src/carve_and_rerun.py", mv[q].filtered_dir)

./src/exper_wizard.py manager -e verify --total-parts 12 -i  data/projs/singleton_cb725a5d7b/base.z3 --clear
./src/analysis_wizard.py singleton -e verify -s z3_4_13_0 -i  data/projs/singleton_cb725a5d7b/base.z3
python3 src/carve_and_rerun.py data/projs/singleton_cb725a5d7b.filtered/base.z3
./src/exper_wizard.py manager -e verify --total-parts 12 -i  data/projs/singleton_1bcd3aa5cb/base.z3 --clear
./src/analysis_wizard.py singleton -e verify -s z3_4_13_0 -i  data/projs/singleton_1bcd3aa5cb/base.z3
python3 src/carve_and_rerun.py data/projs/singleton_1bcd3aa5cb.filtered/base.z3
./src/exper_wizard.py manager -e verify --total-parts 12 -i  data/projs/singleton_d75368331e/base.z3 --clear
./src/analysis_wizard.py singleton -e verify -s z3_4_13_0 -i  data/projs/singleton_d75368331e/base.z3
python3 src/carve_and_rerun.py data/projs/singleton_d75368331e.filtered/base.z3
./src/exper_wizard.py manager -e verify --total-parts 12 -i  data/projs/singleton_b182e2a043/base.z3 --clear
./src/analysis_wiz

In [9]:
print(len(mv.fixable)/len(mv.status[DebuggerStatus.FINISHED]))

# bad_min_ranks = dict()

# for q in mv.fixable:
#     r = mv.reviewers[q]
#     report = r.build_report()
#     indices = report.freq.loc[report.freq["qname"].isin(report.stabilized["qname"])].index
#     report.freq["rank"] = report.freq["trace_count"].rank(method='min', ascending=False)
#     min_rank = report.freq.loc[indices]["rank"].min()
#     max_rank = report.freq.loc[indices]["rank"].max()
#     if min_rank <= 10:
#         continue
#     bad_min_ranks[q] = (min_rank, len(indices))

# for q, (rank, feasible)  in bad_min_ranks.items():
#     print(int(rank), feasible, q)

0.7441860465116279


In [ ]:
for q in mv.status[DebuggerStatus.NO_PROOF]:
    print('"' + mv[q].name_hash + '"', end=", ")

In [ ]:
dirs_to_send = []
additional = []

for q in mv.fixable:
    r = mv.reviewers[q]
    if r.name_hash in LIKELY_FIXED:
        continue
    r.collect_garbage()
    dirs_to_send += r.get_dirs_to_sync()
    additional += f"'{r.name_hash}'"

print("#" + ", ".join(additional))

for i, d in enumerate(dirs_to_send):
    if i % 5 == 0:
        print("echo '", (i //5 +1), "/", len(dirs_to_send)//5, "'")
    print("rsync -avz", d, "g2001:/home/yizhou7/mariposa/", "--inplace", "--delete", "--relative")